In [ ]:
from mpi4py import MPI
import dolfinx as df
import dolfinx.fem.petsc
import numpy as np
import ufl
import matplotlib.pyplot as pl

In [ ]:
def solve_poisson_1d(ne, p=1):
    """
    A python function to solve a one-dimensional Poisson problem
    on a unit interval domain.
    Parameters:
      * ne - number of elements
      * p  - polynomial order of the solution function space
    """
    mesh = df.mesh.create_unit_interval(MPI.COMM_WORLD, ne)

    V = df.fem.functionspace(mesh, ("Lagrange", p))

    T_a = ufl.TrialFunction(V)
    T_t = ufl.TestFunction(V)

    def boundary(x):
        return np.isclose(x[0], 0)
    boundary_dofs = df.fem.locate_dofs_geometrical(V, boundary)
    gD = df.fem.Constant(mesh, df.default_scalar_type(0.0))
    bc = df.fem.dirichletbc(gD, boundary_dofs, V)

    x = ufl.SpatialCoordinate(mesh)
    rhsf = (ufl.pi**2)*ufl.sin(ufl.pi*x[0]/2)/4

    S = ufl.inner(ufl.grad(T_t), ufl.grad(T_a))*ufl.dx
    f = T_t*rhsf*ufl.dx

    problem = df.fem.petsc.LinearProblem(S, f, bcs=[bc], \
                                         petsc_options={"ksp_type": "preonly", \
                                                        "pc_type": "lu"})
    T_i = problem.solve()

    return T_i

In [ ]:
T_P1 = solve_poisson_1d(4)

In [ ]:
T_P1.x.array

In [ ]:
T_P2 = solve_poisson_1d(4, p=2)

In [ ]:
T_P2.x.array

In [ ]:
def evaluate_error(T_i):
    """
    A python function to evaluate the l2 norm of the error in 
    the one dimensional Poisson problem given a known analytical
    solution.
    """
    # Define the exact solution
    x  = ufl.SpatialCoordinate(T_i.function_space.mesh)
    Te = ufl.sin(ufl.pi*x[0]/2)
    
    # Define the error between the exact solution and the given
    # approximate solution
    l2err = df.fem.assemble_scalar(df.fem.form((T_i - Te)*(T_i - Te)*ufl.dx))**0.5
    
    # Return the l2 norm of the error
    return l2err

In [ ]:
# Open a figure for plotting
fig = pl.figure()

# List of polynomial orders to try
ps = [1, 2]
# List of resolutions to try
nelements = [10, 20, 40, 80, 160, 320]
# Keep track of whether we get the expected order of convergence
test_passes = True
# Loop over the polynomial orders
for p in ps:
    # Accumulate the errors
    errors_l2_a = []
    # Loop over the resolutions
    for ne in nelements:
        # Solve the 1D Poisson problem
        T_i = solve_poisson_1d(ne, p)
        # Evaluate the error in the approximate solution
        l2error = evaluate_error(T_i)
        # Print to screen and save
        print('ne = ', ne, ', l2error = ', l2error)
        errors_l2_a.append(l2error)

    # Work out the order of convergence at this p
    hs = 1./np.array(nelements)/p
    
    # Write the errors to disk
    with open('1d_poisson_convergence_p{}.csv'.format(p), 'w') as f:
        np.savetxt(f, np.c_[nelements, hs, errors_l2_a], delimiter=',', 
                   header='nelements, hs, l2errs')
    
    # Fit a line to the convergence data
    fit = np.polyfit(np.log(hs), np.log(errors_l2_a),1)
    print("***********  order of accuracy p={}, order={:.2f}".format(p,fit[0]))
    
    # log-log plot of the error  
    pl.loglog(hs,errors_l2_a,'o-',label='p={}, order={:.2f}'.format(p,fit[0]))
    
    # Test if the order of convergence is as expected
    test_passes = test_passes and fit[0] > p+0.9

# Tidy up the plot
pl.xlabel('h')
pl.ylabel('||e||_2')
pl.grid()
pl.title('Convergence')
pl.legend()
pl.savefig('1d_poisson_convergence.pdf')

print("***********  convergence figure in poisson_convergence.pdf")
# Check if we passed the test
assert(test_passes)
